En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import datetime

In [16]:
class Extraccion:

    def __init__(self):

        self = self
        
    def api_nacional(self, año_inicio, año_final, df_vacio):

        self.año_inicio = año_inicio
        self.año_final = año_final
        self.df_vacio = df_vacio

        for año in range(año_inicio, año_final):
            
            response = requests.get(f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year')

            response.status_code

            response.reason

            df_renovables = pd.json_normalize(response.json()['included'][0]['attributes']['values'])
            df_renovables ['tipo_energía'] = "Renovable"

            df_vacio = pd.concat([df_vacio, df_renovables], axis = 0)

            df_no_renovables = pd.json_normalize(response.json()['included'][1]['attributes']['values']) 
            df_no_renovables ['tipo_energía'] = "No_renovable"

            df_vacio = pd.concat([df_vacio, df_no_renovables], axis = 0)
            
        return df_vacio
    

    def api_ccaa(self,año_inicio,año_final, diccionario, df_vacio):

        self.diccionario = diccionario
        self.año_inicio = año_inicio
        self.año_final = año_final
        self.df_vacio = df_vacio

        for año in range(año_inicio,año_final):

            for k, v in diccionario.items():
                    
                response = requests.get(f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}')

                response.status_code
                
                response.reason
            
                if len(response.json()['included']) == 2:
                    renovables_ccaa = pd.json_normalize(response.json()['included'][0]['attributes']['values']) #renovables
                    renovables_ccaa ['tipo_energía'] = "Renovable"
                    renovables_ccaa ['comunidad'] = k 
                    renovables_ccaa ['id_comunidad'] = v
                
                    df_vacio = pd.concat([df_vacio, renovables_ccaa], axis = 0)

                    no_renovables_ccaa = pd.json_normalize(response.json()['included'][1]['attributes']['values']) #excepción
                    no_renovables_ccaa ['tipo_energía'] = "No_renovable"
                    no_renovables_ccaa ['comunidad'] = k 
                    no_renovables_ccaa ['id_comunidad'] = v

                    df_vacio = pd.concat([df_vacio, no_renovables_ccaa], axis = 0)

                else:
                    no_renovables_ccaa1 = pd.json_normalize(response.json()['included'][0]['attributes']['values']) #excepción
                    no_renovables_ccaa1 ['tipo_energía'] = "No_renovable"
                    no_renovables_ccaa1 ['comunidad'] = k 
                    no_renovables_ccaa1 ['id_comunidad'] = v

                    df_vacio = pd.concat([df_vacio, no_renovables_ccaa1], axis = 0)

        return df_vacio


In [12]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [18]:
df = pd.DataFrame()

In [19]:
api = Extraccion( )
api

In [20]:
api.api_nacional(2011, 2012, df)

,value,percentage,datetime,tipo_energía
0,8.666705e+07,0.310241,2011-01-01T00:00:00.000+01:00,Renovable
0,1.926868e+08,0.689759,2011-01-01T00:00:00.000+01:00,No_renovable


In [22]:
help(Extraccion())

Help on Extraccion in module __main__ object:

class Extraccion(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  api_ccaa(self, año_inicio, año_final, diccionario, df_vacio)
 |  
 |  api_nacional(self, año_inicio, año_final, df_vacio)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [21]:
df2 = pd.DataFrame()

In [23]:
api.api_ccaa(2011, 2012, cod_comunidades, df2)

,value,percentage,datetime,tipo_energía,comunidad,id_comunidad
0,2.029729e+05,1.000000,2011-01-01T00:00:00.000+01:00,No_renovable,Ceuta,8744
0,3.589082e+03,0.016703,2011-01-01T00:00:00.000+01:00,Renovable,Melilla,8745
0,2.112900e+05,0.983297,2011-01-01T00:00:00.000+01:00,No_renovable,Melilla,8745
0,1.099692e+07,0.284857,2011-01-01T00:00:00.000+01:00,Renovable,Andalucía,4
0,2.760811e+07,0.715143,2011-01-01T00:00:00.000+01:00,No_renovable,Andalucía,4
0,7.005216e+06,0.386497,2011-01-01T00:00:00.000+01:00,Renovable,Aragón,5
0,1.111968e+07,0.613503,2011-01-01T00:00:00.000+01:00,No_renovable,Aragón,5
0,3.385157e+05,0.170045,2011-01-01T00:00:00.000+01:00,Renovable,Cantabria,6
0,1.652228e+06,0.829955,2011-01-01T00:00:00.000+01:00,No_renovable,Cantabria,6
0,9.874557e+06,0.410912,2011-01-01T00:00:00.000+01:00,Renovable,Castilla - La Mancha,7


In [ ]:
class Limpieza:
    